In [43]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as rmse, r2_score

from xgboost import XGBRegressor

In [33]:
input_path = "C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data/preprocessed_data_submission_xgboost.pkl"

with open(input_path, "rb") as f:
    data_dict = pkl.load(f)

df = data_dict['data']
df['power'] = data_dict['input_data']['y_true']
df = df.reset_index(drop=True)
df['quarter'] = df['quarter'].apply(lambda x: 'Qtr' + str(x))
df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2020,Qtr1,a19bbfeafb,2aae39f89f,4.434074,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,core,0,2.0,1.0,10
1,2020,Qtr1,a19bbfeafb,3051a7f954,0.162649,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,Unknown,0,2.0,1.0,10
2,2020,Qtr1,a19bbfeafb,45b1ae2725,0.471728,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,super premium,0,2.0,1.0,10
3,2020,Qtr1,a19bbfeafb,599c19172e,0.407395,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,Unknown,0,2.0,1.0,10
4,2020,Qtr1,a19bbfeafb,60c48a7d91,0.748121,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,Unknown,0,2.0,1.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617,2024,Qtr2,aa30935544,e08e116534,1.173713,0.339480,0.319083,0.353705,0.021500,0.063802,...,0.0,0.000893,0.017360,0.006173,0.046728,Unknown,49,1.0,0.0,43
1618,2024,Qtr2,aa30935544,e7286b8344,0.819116,0.339480,0.319083,0.353705,0.079195,0.153498,...,0.0,0.000252,0.000797,0.000253,0.052988,premium,49,1.0,0.0,43
1619,2024,Qtr2,aa30935544,f62a3510a9,3.851052,0.339480,0.319083,0.353705,0.021500,0.063802,...,0.0,0.000893,0.017360,0.006173,0.046728,Unknown,49,1.0,0.0,43
1620,2024,Qtr2,aa30935544,f6bb5e2016,1.153945,0.339480,0.319083,0.353705,0.021500,0.063802,...,0.0,0.000893,0.017360,0.006173,0.046728,Unknown,49,1.0,0.0,43


In [25]:
test_path = 'C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data/preprocessed_data_submission_format_category.pkl'
with open(test_path, 'rb') as f:
    test_dict = pkl.load(f)

df_reference = test_dict['df_reference']
df_reference['quarter'] = df_reference['quarter'].apply(lambda x: 'Qtr' + str(x))
df_reference.drop(columns=['power'], inplace=True)
df_reference

,year,quarter,country,brand,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,Other Traditional Mktg,...,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id,country_id,brand_id,category_id
0,2024,Qtr3,653b4e6d12,038baa6798,0.331739,0.181843,-0.163830,0.040606,0.097200,0.168082,...,0.041222,0.427781,Unknown,42.0,0.0,1.0,5.949249,0.0,1.0,0.0
1,2024,Qtr4,653b4e6d12,038baa6798,0.497000,0.386958,0.212658,0.040606,0.097200,0.168082,...,0.039880,0.523762,Unknown,43.0,1.0,2.0,7.029329,0.0,1.0,0.0
2,2025,Qtr1,653b4e6d12,038baa6798,0.632197,0.282923,0.650972,0.040606,0.097200,0.168082,...,0.085111,0.160916,Unknown,52.0,2.0,1.0,8.031429,0.0,1.0,0.0
3,2025,Qtr2,653b4e6d12,038baa6798,0.349474,0.351890,0.118628,0.040606,0.097200,0.168082,...,-0.007558,0.297833,Unknown,53.0,1.0,0.0,8.982841,0.0,1.0,0.0
4,2024,Qtr3,653b4e6d12,05e5aaf0f4,0.389718,0.354222,-0.104215,0.000000,0.000000,0.269244,...,0.009328,0.659810,premium,42.0,0.0,1.0,6.116995,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2025,Qtr2,aa30935544,f6bb5e2016,0.311643,0.129005,0.322614,0.021500,0.063802,0.000578,...,0.084775,-0.065945,Unknown,53.0,1.0,0.0,43.010378,2.0,95.0,0.0
452,2024,Qtr3,aa30935544,fa0fa449fa,0.461871,0.216491,0.065886,0.021500,0.063802,0.000578,...,-0.147824,0.047623,Unknown,42.0,0.0,1.0,39.800794,2.0,98.0,0.0
453,2024,Qtr4,aa30935544,fa0fa449fa,0.160945,0.129005,0.214129,0.021500,0.063802,0.000578,...,-0.112948,0.029606,Unknown,43.0,1.0,2.0,40.925496,2.0,98.0,0.0
454,2025,Qtr1,aa30935544,fa0fa449fa,0.084360,-0.088032,0.332329,0.021500,0.063802,0.000578,...,0.073488,-0.006811,Unknown,52.0,2.0,1.0,42.037807,2.0,98.0,0.0


In [34]:
format_path = r'C:\Users\40107904\OneDrive - Anheuser-Busch InBev\ABI\WORK\hackathon_power\hackathon_lt_equity\data\lte_participants_data\BG_Data_Hackathon_Test_Predict_masked.csv'
format = pd.read_csv(format_path)
format = format.merge(df_reference, left_on=['Country', 'Brand', 'Year', 'Quarter'], right_on=['country', 'brand', 'year', 'quarter'], how='outer')
format.drop(columns=['Country', 'Brand', 'Year', 'Quarter', 'Predicted Power', 'brand_id', 'country_id', 'category_id'], inplace=True)
format

,year,quarter,country,brand,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,Other Traditional Mktg,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2024,Qtr3,653b4e6d12,038baa6798,0.331739,0.181843,-0.163830,0.040606,0.097200,0.168082,...,0.235124,-0.045122,-0.058675,0.041222,0.427781,Unknown,42.0,0.0,1.0,5.949249
1,2024,Qtr4,653b4e6d12,038baa6798,0.497000,0.386958,0.212658,0.040606,0.097200,0.168082,...,0.235124,0.024093,-0.075142,0.039880,0.523762,Unknown,43.0,1.0,2.0,7.029329
2,2025,Qtr1,653b4e6d12,038baa6798,0.632197,0.282923,0.650972,0.040606,0.097200,0.168082,...,0.235124,-0.059459,-0.155857,0.085111,0.160916,Unknown,52.0,2.0,1.0,8.031429
3,2025,Qtr2,653b4e6d12,038baa6798,0.349474,0.351890,0.118628,0.040606,0.097200,0.168082,...,0.235124,-0.138515,-0.242971,-0.007558,0.297833,Unknown,53.0,1.0,0.0,8.982841
4,2024,Qtr3,653b4e6d12,05e5aaf0f4,0.389718,0.354222,-0.104215,0.000000,0.000000,0.269244,...,0.353467,-0.119348,-0.222845,0.009328,0.659810,premium,42.0,0.0,1.0,6.116995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2025,Qtr2,aa30935544,f6bb5e2016,0.311643,0.129005,0.322614,0.021500,0.063802,0.000578,...,0.000000,-0.164393,-0.011480,0.084775,-0.065945,Unknown,53.0,1.0,0.0,43.010378
452,2024,Qtr3,aa30935544,fa0fa449fa,0.461871,0.216491,0.065886,0.021500,0.063802,0.000578,...,0.000000,-0.117452,-0.038195,-0.147824,0.047623,Unknown,42.0,0.0,1.0,39.800794
453,2024,Qtr4,aa30935544,fa0fa449fa,0.160945,0.129005,0.214129,0.021500,0.063802,0.000578,...,0.000000,-0.037113,-0.261559,-0.112948,0.029606,Unknown,43.0,1.0,2.0,40.925496
454,2025,Qtr1,aa30935544,fa0fa449fa,0.084360,-0.088032,0.332329,0.021500,0.063802,0.000578,...,0.000000,-0.096967,-0.140439,0.073488,-0.006811,Unknown,52.0,2.0,1.0,42.037807


In [35]:
final_df = pd.concat([df, format], axis=0   , ignore_index=True)
final_df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2020,Qtr1,a19bbfeafb,2aae39f89f,4.434074,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,core,0.0,2.0,1.0,10.000000
1,2020,Qtr1,a19bbfeafb,3051a7f954,0.162649,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,Unknown,0.0,2.0,1.0,10.000000
2,2020,Qtr1,a19bbfeafb,45b1ae2725,0.471728,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,super premium,0.0,2.0,1.0,10.000000
3,2020,Qtr1,a19bbfeafb,599c19172e,0.407395,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,Unknown,0.0,2.0,1.0,10.000000
4,2020,Qtr1,a19bbfeafb,60c48a7d91,0.748121,0.380381,0.321212,0.615772,0.036739,0.068597,...,0.0,0.187635,0.248801,0.259301,0.314381,Unknown,0.0,2.0,1.0,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,2025,Qtr2,aa30935544,f6bb5e2016,NaN,0.311643,0.129005,0.322614,0.021500,0.063802,...,0.0,-0.164393,-0.011480,0.084775,-0.065945,Unknown,53.0,1.0,0.0,43.010378
2074,2024,Qtr3,aa30935544,fa0fa449fa,NaN,0.461871,0.216491,0.065886,0.021500,0.063802,...,0.0,-0.117452,-0.038195,-0.147824,0.047623,Unknown,42.0,0.0,1.0,39.800794
2075,2024,Qtr4,aa30935544,fa0fa449fa,NaN,0.160945,0.129005,0.214129,0.021500,0.063802,...,0.0,-0.037113,-0.261559,-0.112948,0.029606,Unknown,43.0,1.0,2.0,40.925496
2076,2025,Qtr1,aa30935544,fa0fa449fa,NaN,0.084360,-0.088032,0.332329,0.021500,0.063802,...,0.0,-0.096967,-0.140439,0.073488,-0.006811,Unknown,52.0,2.0,1.0,42.037807


# XG Boost

In [46]:
# Assuming final_df is already loaded
# Target variable
target = "power"

train = final_df[final_df[target].notnull()]
test = final_df[final_df[target].isnull()]

X_train = train.drop(columns=[target])
y_train = train[target]
X_test = test.drop(columns=[target])
y_test = test[target]  # This will be all NaNs

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
numerical_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Preprocessing for categorical & numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols)
    ]
)

# Define the XGBoost regressor
xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# Build pipeline
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb)
])


# Fit model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)


In [48]:
print(y_pred)

[ 1.2628958   1.2203312   1.1647062   1.0385792   1.4521257   2.5706947
  2.441752    1.3223457   3.1725998   0.9835078   0.8836757   0.8058936
  0.70018655  2.4977875   0.72239655  0.6911893   0.9702187   0.5822929
  0.7553937   0.67017365  2.2252061   3.8011982   2.638756    3.401376
  0.8058936   2.6230464   2.7294872   0.9301566   0.9454801   0.8136631
  2.6392145   2.9426556   4.3892665   3.414142    5.629998    3.612428
  2.8506799   2.6212778   0.70461875  0.7334376   0.52885425  0.5437465
  2.3817925   0.5989062   0.76334244  1.0743212   0.6251814   0.76334244
  0.8099597   0.821097    0.6371308   0.73044425  3.4448984   3.5478222
  3.5942519   3.9209821   0.78483605  0.79054064  0.96050775  0.85616726
  0.7742731   0.6982913   0.6011723   0.68905306  0.69431335  0.77019435
  0.77019435  0.70001787  9.856708   10.532739   12.188429   12.815812
  0.9618509   2.6108177   2.5411172   0.9618509   2.2557068   1.8525432
  1.8487377   1.9615121   2.7591298   1.2983557   0.70860475  2.

In [52]:
X_test['Predicted Power'] = y_pred
X_test

,year,quarter,country,brand,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,Other Traditional Mktg,...,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id,Predicted Power
1622,2024,Qtr3,653b4e6d12,038baa6798,0.331739,0.181843,-0.163830,0.040606,0.097200,0.168082,...,-0.045122,-0.058675,0.041222,0.427781,Unknown,42.0,0.0,1.0,5.949249,1.262896
1623,2024,Qtr4,653b4e6d12,038baa6798,0.497000,0.386958,0.212658,0.040606,0.097200,0.168082,...,0.024093,-0.075142,0.039880,0.523762,Unknown,43.0,1.0,2.0,7.029329,1.220331
1624,2025,Qtr1,653b4e6d12,038baa6798,0.632197,0.282923,0.650972,0.040606,0.097200,0.168082,...,-0.059459,-0.155857,0.085111,0.160916,Unknown,52.0,2.0,1.0,8.031429,1.164706
1625,2025,Qtr2,653b4e6d12,038baa6798,0.349474,0.351890,0.118628,0.040606,0.097200,0.168082,...,-0.138515,-0.242971,-0.007558,0.297833,Unknown,53.0,1.0,0.0,8.982841,1.038579
1626,2024,Qtr3,653b4e6d12,05e5aaf0f4,0.389718,0.354222,-0.104215,0.000000,0.000000,0.269244,...,-0.119348,-0.222845,0.009328,0.659810,premium,42.0,0.0,1.0,6.116995,1.452126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,2025,Qtr2,aa30935544,f6bb5e2016,0.311643,0.129005,0.322614,0.021500,0.063802,0.000578,...,-0.164393,-0.011480,0.084775,-0.065945,Unknown,53.0,1.0,0.0,43.010378,1.917519
2074,2024,Qtr3,aa30935544,fa0fa449fa,0.461871,0.216491,0.065886,0.021500,0.063802,0.000578,...,-0.117452,-0.038195,-0.147824,0.047623,Unknown,42.0,0.0,1.0,39.800794,1.238082
2075,2024,Qtr4,aa30935544,fa0fa449fa,0.160945,0.129005,0.214129,0.021500,0.063802,0.000578,...,-0.037113,-0.261559,-0.112948,0.029606,Unknown,43.0,1.0,2.0,40.925496,1.252248
2076,2025,Qtr1,aa30935544,fa0fa449fa,0.084360,-0.088032,0.332329,0.021500,0.063802,0.000578,...,-0.096967,-0.140439,0.073488,-0.006811,Unknown,52.0,2.0,1.0,42.037807,2.061448


# Output

In [74]:
format = X_test[['country', 'brand', 'year', 'quarter', 'Predicted Power']]
format.rename(columns={'country': 'Country', 'brand': 'Brand', 'year': 'Year', 'quarter': 'Quarter', 'Predicted Power': 'power_pred'}, inplace=True)
format

C:\Users\40107904\AppData\Local\Temp\ipykernel_20888\3434450507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  format.rename(columns={'country': 'Country', 'brand': 'Brand', 'year': 'Year', 'quarter': 'Quarter', 'Predicted Power': 'power_pred'}, inplace=True)


,Country,Brand,Year,Quarter,power_pred
1622,653b4e6d12,038baa6798,2024,Qtr3,1.262896
1623,653b4e6d12,038baa6798,2024,Qtr4,1.220331
1624,653b4e6d12,038baa6798,2025,Qtr1,1.164706
1625,653b4e6d12,038baa6798,2025,Qtr2,1.038579
1626,653b4e6d12,05e5aaf0f4,2024,Qtr3,1.452126
...,...,...,...,...,...
2073,aa30935544,f6bb5e2016,2025,Qtr2,1.917519
2074,aa30935544,fa0fa449fa,2024,Qtr3,1.238082
2075,aa30935544,fa0fa449fa,2024,Qtr4,1.252248
2076,aa30935544,fa0fa449fa,2025,Qtr1,2.061448


In [75]:
format.groupby(['Country', 'Year', 'Quarter']).agg({'power_pred': 'sum'}).reset_index()
# normalize power by each country x year x quarter
normalize_c = format.groupby(['Country', 'Year', 'Quarter']).agg({'power_pred': 'sum'}).reset_index()
format = format.merge(normalize_c, left_on=['Country', 'Year', 'Quarter'], right_on=['Country', 'Year', 'Quarter'], suffixes=('', '_total'))
format

,Country,Brand,Year,Quarter,power_pred,power_pred_total
0,653b4e6d12,038baa6798,2024,Qtr3,1.262896,106.687416
1,653b4e6d12,038baa6798,2024,Qtr4,1.220331,93.883064
2,653b4e6d12,038baa6798,2025,Qtr1,1.164706,103.701134
3,653b4e6d12,038baa6798,2025,Qtr2,1.038579,91.868721
4,653b4e6d12,05e5aaf0f4,2024,Qtr3,1.452126,106.687416
...,...,...,...,...,...,...
451,aa30935544,f6bb5e2016,2025,Qtr2,1.917519,115.977844
452,aa30935544,fa0fa449fa,2024,Qtr3,1.238082,107.872688
453,aa30935544,fa0fa449fa,2024,Qtr4,1.252248,110.559868
454,aa30935544,fa0fa449fa,2025,Qtr1,2.061448,113.015236


In [76]:
format['power_pred'] = format['power_pred'] / format['power_pred_total'] * 100
format

,Country,Brand,Year,Quarter,power_pred,power_pred_total
0,653b4e6d12,038baa6798,2024,Qtr3,1.183735,106.687416
1,653b4e6d12,038baa6798,2024,Qtr4,1.299842,93.883064
2,653b4e6d12,038baa6798,2025,Qtr1,1.123137,103.701134
3,653b4e6d12,038baa6798,2025,Qtr2,1.130504,91.868721
4,653b4e6d12,05e5aaf0f4,2024,Qtr3,1.361103,106.687416
...,...,...,...,...,...,...
451,aa30935544,f6bb5e2016,2025,Qtr2,1.653350,115.977844
452,aa30935544,fa0fa449fa,2024,Qtr3,1.147725,107.872688
453,aa30935544,fa0fa449fa,2024,Qtr4,1.132643,110.559868
454,aa30935544,fa0fa449fa,2025,Qtr1,1.824044,113.015236


In [77]:
format_path = r'C:\Users\40107904\OneDrive - Anheuser-Busch InBev\ABI\WORK\hackathon_power\hackathon_lt_equity\data\lte_participants_data\BG_Data_Hackathon_Test_Predict_masked.csv'
format_out = pd.read_csv(format_path)
format_out = format_out.merge(format[['Country', 'Brand', 'Year', 'Quarter','power_pred']], left_on=['Country', 'Brand', 'Year', 'Quarter'], right_on=['Country', 'Brand', 'Year', 'Quarter'], how='left')
format_out

,Year,Quarter,Country,Brand,Predicted Power,power_pred
0,2024,Qtr3,a19bbfeafb,cd46a210c6,NaN,9.440711
1,2024,Qtr3,a19bbfeafb,b6a118bf97,NaN,7.299715
2,2024,Qtr3,a19bbfeafb,53a67488ad,NaN,0.682417
3,2024,Qtr3,a19bbfeafb,a0bc54a71d,NaN,3.093486
4,2024,Qtr3,a19bbfeafb,8.55E+09,NaN,3.828943
...,...,...,...,...,...,...
99,2025,Qtr2,aa30935544,96ddb9088d,NaN,1.891470
100,2025,Qtr2,aa30935544,6e31ce004a,NaN,4.147028
101,2025,Qtr2,aa30935544,71e9e5f0c0,NaN,1.241764
102,2025,Qtr2,aa30935544,a5f7e77993,NaN,1.981403


In [78]:
format_out['Predicted Power'] = format_out['power_pred']
format_out

,Year,Quarter,Country,Brand,Predicted Power,power_pred
0,2024,Qtr3,a19bbfeafb,cd46a210c6,9.440711,9.440711
1,2024,Qtr3,a19bbfeafb,b6a118bf97,7.299715,7.299715
2,2024,Qtr3,a19bbfeafb,53a67488ad,0.682417,0.682417
3,2024,Qtr3,a19bbfeafb,a0bc54a71d,3.093486,3.093486
4,2024,Qtr3,a19bbfeafb,8.55E+09,3.828943,3.828943
...,...,...,...,...,...,...
99,2025,Qtr2,aa30935544,96ddb9088d,1.891470,1.891470
100,2025,Qtr2,aa30935544,6e31ce004a,4.147028,4.147028
101,2025,Qtr2,aa30935544,71e9e5f0c0,1.241764,1.241764
102,2025,Qtr2,aa30935544,a5f7e77993,1.981403,1.981403


In [79]:
format_out.groupby(['Country', 'Year', 'Quarter']).agg({'Predicted Power': 'sum'}).reset_index()

,Country,Year,Quarter,Predicted Power
0,653b4e6d12,2024,Qtr3,38.313709
1,653b4e6d12,2024,Qtr4,43.192249
2,653b4e6d12,2025,Qtr1,34.366821
3,653b4e6d12,2025,Qtr2,38.582134
4,a19bbfeafb,2024,Qtr3,48.579094
5,a19bbfeafb,2024,Qtr4,55.355942
6,a19bbfeafb,2025,Qtr1,43.798832
7,a19bbfeafb,2025,Qtr2,43.088043
8,aa30935544,2024,Qtr3,19.630230
9,aa30935544,2024,Qtr4,17.556541


In [68]:
format_out.to_csv('../submissions/submission_xgboost.csv')